## Running XULE in Arelle  
This interactive environment copies the [free, open-source XULE plugin](https://github.com/xbrlus/xule/) to a working Python environment with [Arelle](https://pypi.org/project/arelle-release/) plus [additional packages](https://raw.githubusercontent.com/xbrlus/xule/jupyter/environment.yml). Click the code cell below (gray background) to bring it in focus, then click the run button to execute the install steps. **This step only needs to be completed once for the notebook session** unless different versions of Arelle or XULE are required.

In [ ]:
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.30.11
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist 
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = site.getsitepackages()[0] + '/arelle/plugin/xule/'
temp = site.getsitepackages()[0] + '/arelle/plugin/temp/'
tempxule = site.getsitepackages()[0] + '/arelle/plugin/temp/plugin/xule/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            shutil.rmtree(temp)
            shutil.rmtree(xuledir)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory 
!git clone --quiet --depth=1 --branch 30041 --single-branch https://github.com/xbrlus/xule.git temp
shutil.move(tempxule, xuledir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v

### 1. Define a XULE Expression
The cell below is a XULE 'file' called taxonomy-hierarchy.xule **which starts on the line below _%%writefile_** - *this is a Jupyter-specific command* that saves the cell's contents as a file when the cell is run.

The XULE expression in the file creates an Excel spreadsheet of the presentation hierarchy for a filing (defined in the expression as $schema) and saves it in location `/content/` as the file `taxonomy-hierarchy.xlsx`. This location (as well as references to it in step 2) assumes Colab is the environment, and should be adjusted based on local machine or online service location for storage.

Each tab in the spreadsheet is an array corresponding to a section of the presentation linkbase (ie. the financial statements and its disclosures) The function .agg-to-dict(1) is applied to each different set of 6 characters produced by the list (and printed to the first column) to create spreadsheet tabs.

The columns in each worksheet (1 - 9) include details from the presentation linkbase. To create a visual hierarchy, an `if/else condition` evaluates each concept's navigation-depth and preferred-label and applies blank spaces to the presentation label within the statement or disclosure's strucuture.

The `/** commented  US GAAP 2024 entry point**/` in the expression (or another XBRL report URL) can replace the current report URL as $schema to produce an Excel of the entire taxonomy published by the Finanacial Accounting Standards Board (a 'base' taxonomy for US GAAP filers) or any other XBRL report presentation hierarchy.

The `/** commented  lines to output as .json**/` in the expression creates the arrays in this format (remember to comment out the .xlsx lines)

Click the code cell below then **use 'Run All Below' from Jupyter's Cell menu** to save and run XULE with a single click (NB: after the cell is run once, toggling **%%writefile** to **%load** will display the contents of the sample.xule file in the cell).

### 2. Run XULE against an XBRL report

In [ ]:
!arelleCmdLine --plugins 'xodel|transforms/SEC|validate/EFM|inlineXbrlDocumentSet' -f https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/0001079973-24-001419-xbrl.zip --xule-rule-set https://raw.githubusercontent.com/xbrlus/xule/jupyter/2024-UGT-norm.zip --xule-run-only createInstance,make_cf_fact_values,make_bs_fact_values,make_FA,make_OA,make_AP,make_AE,make_LTDC,make_OLC,make_UCL,make_LTDNC,make_OLNC,make_LE,make_is_fact_values,make_standard_data,get_dei_data,get_filing_data,get_bus_desc,get_tax_data,get_royalty_data,get_marketing,make_STDC,get_trading_symbol --xodel-location /content/ --xince-file-type xml --xule-arg format=xml --xule-arg InstanceName=normalized --xule-arg FILING_DATE=2024-10-17 --xule-arg SIC_CODE=1311 --xule-arg FILING_ACCESSION=0001079973-24-001419 --xule-arg FILING_URL=http://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm --xule-arg TAXONOMY_LOC=https://taxonomies.xbrl.us/standardized-statements/xusss.xsd --httpUserAgent='XULE-Arelle (xbrl.us; support@xbrl.us)'


The commands in the cell above:
1. uses the XULE plugin to compile the XULE file from step 1 into a .zip file (XULE must be compiled before it will run)
1. copies the source .xule and compiled .zip files to the content folder in the environment (click the folder at left to download), and
1. uses XULE and other plugins to run the .zip against an XBRL report listed in the command (the SEC requires a User Agent to access reports programmatically), create a file according to the XULE expression (.xlsx or .json) and generate a log.xml of the process. 

The XULE expression and Arelle commands define `/content/` as the location for files - XULE, .zip, .xlsx and .xml - so these outputs will appear in the folder at left (default Colab environment) after processing completes. 

If the `$schema` constant in the expression is updated for a 'base' taxonomy (ie. FASB, FERC, IFRS), a report - also known as an instance document - using the same base taxonomy must be used with the last command in the sequence.

(NB: run the first code cell in this notebook with the -h switch instead of -v to get configuration paramters for the command).
